<a href="https://colab.research.google.com/github/yingzibu/drug_design_JAK/blob/main/drl_unit1_ppo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt install swig cmake --quiet
!pip install -r https://raw.githubusercontent.com/huggingface/deep-rl-class/main/notebooks/unit1/requirements-unit1.txt --quiet

In [ ]:
!sudo apt-get update --quiet
!apt install python-opengl --quiet
!apt install ffmpeg --quiet
!apt install xvfb --quiet
!pip3 install pyvirtualdisplay --quiet

In [5]:
from pyvirtualdisplay import Display

virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

In [6]:
import warnings
from tqdm import tqdm
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [7]:
import gymnasium

from huggingface_sb3 import load_from_hub, package_to_hub
from huggingface_hub import notebook_login # To log to our Hugging Face account to be able to upload models to the Hub.

from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor

/usr/local/lib/python3.10/dist-packages/torch/utils/tensorboard/__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, "__version__") or LooseVersion(


In [8]:
import gymnasium as gym
env = gym.make('LunarLander-v2')

obs, info = env.reset()
for _ in tqdm(range(10000)):
    rand_action = env.action_space.sample()
    # print('action taken:', rand_action)
    obs, r, ter, tru, info = env.step(rand_action)
    if ter or tru:
        print('env reset')
        obs, info = env.reset()
        break


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
  1%|▏         | 125/10000 [00:00<00:00, 11026.50it/s]

env reset


In [9]:
env.reset()
env.observation_space.shape, env.observation_space.sample()

((8,),
 array([75.068245  , 69.780914  , -1.7165196 ,  2.3593028 , -2.36692   ,
        -2.4154074 ,  0.9821021 ,  0.79290885], dtype=float32))

In [10]:
env.reset()
env.action_space.n, env.action_space.sample()

(4, 1)

In [11]:
env = make_vec_env('LunarLander-v2', n_envs=16)

In [12]:
model = PPO(
    policy = 'MlpPolicy',
    env = env,
    n_steps = 1024,
    batch_size = 64,
    n_epochs = 4,
    gamma = 0.999,
    gae_lambda = 0.98,
    ent_coef = 0.01,
    verbose=1)

Using cuda device


In [18]:
from tqdm.auto import tqdm
from stable_baselines3.common.callbacks import BaseCallback

class ProgressBarCallback(BaseCallback):
    """
    :param pbar: (tqdm.pbar) Progress bar object
    """

    def __init__(self, pbar):
        super().__init__()
        self._pbar = pbar

    def _on_step(self):
        # Update the progress bar:
        self._pbar.n = self.num_timesteps
        self._pbar.update(0)


# this callback uses the 'with' block, allowing for correct initialisation and destruction
class ProgressBarManager(object):
    def __init__(self, total_timesteps):  # init object with total timesteps
        self.pbar = None
        self.total_timesteps = total_timesteps

    def __enter__(self):  # create the progress bar and callback, return the callback
        self.pbar = tqdm(total=self.total_timesteps)

        return ProgressBarCallback(self.pbar)

    def __exit__(self, exc_type, exc_val, exc_tb):  # close the callback
        self.pbar.n = self.total_timesteps
        self.pbar.update(0)
        self.pbar.close()

# model = TD3("MlpPolicy", "Pendulum-v1", verbose=0)


# Using a context manager garanties that the tqdm progress bar closes correctly
# with ProgressBarManager(2000) as callback:
#     model.learn(2000, callback=callback)

In [ ]:
# TODO: Train it for 1,000,000 timesteps
with ProgressBarManager(int(1e7)) as callback:
    model.learn(total_timesteps=int(1e7), callback=callback)
# TODO: Specify file name for model and save the model to file
model_name = "test"
model.save(model_name)

In [24]:
eval_env = Monitor(gym.make('LunarLander-v2'))
mean, std = evaluate_policy(model, eval_env, n_eval_episodes=10,
                            deterministic=True)
print(f'mean_reward={mean}+/-{std}')

mean_reward=280.9481654+/-16.132270534056204


In [25]:
notebook_login()
!git config --global credential.helper store

Let's fill the `package_to_hub` function:
- `model`: our trained model.
- `model_name`: the name of the trained model that we defined in `model_save`
- `model_architecture`: the model architecture we used, in our case PPO
- `env_id`: the name of the environment, in our case `LunarLander-v2`
- `eval_env`: the evaluation environment defined in eval_env
- `repo_id`: the name of the Hugging Face Hub Repository that will be created/updated `(repo_id = {username}/{repo_name})`

💡 **A good name is {username}/{model_architecture}-{env_id}**

- `commit_message`: message of the commit

In [40]:
import gymnasium as gym
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.env_util import make_vec_env
from huggingface_sb3 import package_to_hub

repo_id = 'eikoenchine/ppo-LunarLander-v2'
env_id = "LunarLander-v2"
eval_env = DummyVecEnv([lambda: Monitor(
            gym.make(env_id, render_mode='rgb_array'))])
model_architecture = 'PPO'
commit_message = 'trained 1e7 timesteps'
package_to_hub(model=model_reload,
               model_name=model_name,
               model_architecture=model_architecture,
               env_id=env_id,
               eval_env=eval_env,
               repo_id=repo_id,
               commit_message=commit_message)

ℹ This function will save, evaluate, generate a video of your agent,
create a model card and push everything to the hub. It might take up to 1min.
This is a work in progress: if you encounter a bug, please open an issue.


/usr/local/lib/python3.10/dist-packages/gym/envs/registration.py:440: UserWarning: WARN: The `registry.env_specs` property along with `EnvSpecTree` is deprecated. Please use `registry` directly as a dictionary instead.
  logger.warn(


Saving video to /tmp/tmp8ripmh2n/-step-0-to-step-1000.mp4
Moviepy - Building video /tmp/tmp8ripmh2n/-step-0-to-step-1000.mp4.
Moviepy - Writing video /tmp/tmp8ripmh2n/-step-0-to-step-1000.mp4



Moviepy - Done !
Moviepy - video ready /tmp/tmp8ripmh2n/-step-0-to-step-1000.mp4
ℹ Pushing repo eikoenchine/ppo-LunarLander-v2 to the Hugging Face
Hub


Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

policy.pth:   0%|          | 0.00/43.3k [00:00<?, ?B/s]

policy.optimizer.pth:   0%|          | 0.00/88.1k [00:00<?, ?B/s]

test.zip:   0%|          | 0.00/147k [00:00<?, ?B/s]

ℹ Your model is pushed to the Hub. You can view your model here:
https://huggingface.co/eikoenchine/ppo-LunarLander-v2/tree/main/


'https://huggingface.co/eikoenchine/ppo-LunarLander-v2/tree/main/'

In [28]:
from huggingface_sb3 import load_from_hub
repo_id = repo_id # The repo_id
filename = "test.zip" # The model filename.zip
custom_objects = {
            "learning_rate": 0.0,
            "lr_schedule": lambda _: 0.0,
            "clip_range": lambda _: 0.0,
}
checkpoint = load_from_hub(repo_id, filename)
model = PPO.load(checkpoint, custom_objects=custom_objects,
                 print_system_info=True)

== CURRENT SYSTEM INFO ==
- OS: Linux-5.15.109+-x86_64-with-glibc2.35 # 1 SMP Fri Jun 9 10:57:30 UTC 2023
- Python: 3.10.6
- Stable-Baselines3: 2.0.0a5
- PyTorch: 2.0.1+cu118
- GPU Enabled: True
- Numpy: 1.22.4
- Cloudpickle: 2.2.1
- Gymnasium: 0.28.1
- OpenAI Gym: 0.25.2

== SAVED MODEL SYSTEM INFO ==
- OS: Linux-5.15.109+-x86_64-with-glibc2.35 # 1 SMP Fri Jun 9 10:57:30 UTC 2023
- Python: 3.10.6
- Stable-Baselines3: 2.0.0a5
- PyTorch: 2.0.1+cu118
- GPU Enabled: True
- Numpy: 1.22.4
- Cloudpickle: 2.2.1
- Gymnasium: 0.28.1
- OpenAI Gym: 0.25.2



In [29]:
eval_env = Monitor(gym.make("LunarLander-v2"))
mean_reward, std_reward = evaluate_policy(model, eval_env,
                                          n_eval_episodes=10,
                                          deterministic=True)
print(f"mean_reward={mean_reward:.2f} +/- {std_reward}")

mean_reward=279.01 +/- 18.95655814799715


# train again from pretrained model

In [46]:
env = make_vec_env('LunarLander-v2', n_envs=16)
repo_id = repo_id # The repo_id
filename = "test.zip" # The model filename.zip
# custom_objects = {
#             "learning_rate": 0.01,
#             "lr_schedule": lambda _: 0.01,
#             "clip_range": lambda _: 0.01,
# }
checkpoint = load_from_hub(repo_id, filename)
model = PPO.load(checkpoint,
                 env=env)



In [45]:
with ProgressBarManager(int(1e6)) as callback:
    model.learn(total_timesteps=int(1e6), callback=callback)

# TODO: Specify file name for model and save the model to file
model_name = "test"
model.save(model_name)

eval_env = Monitor(gym.make("LunarLander-v2"))
mean_reward, std_reward = evaluate_policy(model, eval_env,
                                          n_eval_episodes=10,
                                          deterministic=True)
print(f"mean_reward={mean_reward:.2f} +/- {std_reward}")

  0%|          | 0/1000000 [00:00<?, ?it/s]

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 183      |
|    ep_rew_mean     | 286      |
| time/              |          |
|    fps             | 3058     |
|    iterations      | 1        |
|    time_elapsed    | 5        |
|    total_timesteps | 16384    |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 175         |
|    ep_rew_mean          | 279         |
| time/                   |             |
|    fps                  | 2158        |
|    iterations           | 2           |
|    time_elapsed         | 15          |
|    total_timesteps      | 32768       |
| train/                  |             |
|    approx_kl            | 0.004482039 |
|    clip_fraction        | 0.0543      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.458      |
|    explained_variance   | 0.972       |
|    learning_rate        | 0.

In [34]:
repo_id = 'eikoenchine/ppo-LunarLander-v2'
env_id = "LunarLander-v2"
eval_env = DummyVecEnv([lambda: Monitor(
            gym.make(env_id, render_mode='rgb_array'))])
model_architecture = 'PPO'
commit_message = 'trained 1e7 timesteps'
package_to_hub(model=model,
               model_name=model_name,
               model_architecture=model_architecture,
               env_id=env_id,
               eval_env=eval_env,
               repo_id=repo_id,
               commit_message=commit_message)

In [39]:
eval_env = Monitor(gym.make("LunarLander-v2"))
# model_reload = model.load('test', env=env)
mean_reward, std_reward = evaluate_policy(model_reload, eval_env,
                                          n_eval_episodes=10,
                                          deterministic=True)
print(f"mean_reward={mean_reward:.2f} +/- {std_reward}")

mean_reward=293.68 +/- 13.687768584624644


In [38]:
model_reload.learn(100000, )

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 180      |
|    ep_rew_mean     | 287      |
| time/              |          |
|    fps             | 3938     |
|    iterations      | 1        |
|    time_elapsed    | 8        |
|    total_timesteps | 32768    |
---------------------------------
